In [19]:
import xarray
from datetime import datetime, timedelta
from tqdm import trange
import numpy as np
import pandas as pd

def _to_int(date):
    return 10000*date.year + 100*date.month + date.day

def _mask(array, xbounds, ybounds):
    _array = array[0, :, :]
    masked = _array.where(
        np.logical_and(_array.X >= xbounds[0], _array.X <= xbounds[1]).astype(int) + 
        np.logical_and(_array.Y >= ybounds[0], _array.Y <= ybounds[1]).astype(int) == 2, 
        drop=True)
    return masked

def _aggregate(array, n=2, type='mean'):
    assert array.shape[0] % n == 0 and array.shape[1] % n == 0, f"Array must be divisible by {n} in order to aggregate with no left-overs."
    agg = np.zeros((array.shape[0] // n, array.shape[1] // n))
    for i in range(array.shape[0] // n):
        for j in range(array.shape[1] // n):
            if type == 'mean':
                agg[i, j] = array[n*(i):n*(i+1), n*(j):n*(j+1)].mean()
            elif type == 'max':
                agg[i, j] = array[n*(i):n*(i+1), n*(j):n*(j+1)].max()
    return xarray.DataArray(agg, dims=['X', 'Y'], coords={'X': np.linspace(0, array.shape[0]-n, agg.shape[0]), 'Y': np.linspace(0, array.shape[1]-n, agg.shape[1])})

# Points sampled every 1000m. Select 60 x and 60 y per time slice then aggregate grids of 4 (via mean) to make 30 x 30 y.
# Make sure that bounds chosen are central as data availability in non-central regions is sparse (included sea and non-Nordic domiciled regions?)
xbounds = [5e6, 5.06e6]
ybounds = [4.41e6, 4.47e6]
aggtype = 'max'

start = datetime(2018, 1, 1)
for i in trange(365):
    day = start + timedelta(days=i)
    ds = xarray.open_dataset(f'C:/Users/bened/Documents/University/Cambridge/Thesis/Data/NordicDailyPrecip/NGCD_RR_type1_version_22.03_{str(_to_int(day))}.nc')
    precipitation = _mask(ds['RR'], xbounds, ybounds) # (1, X, Y) dimension --> (reduced_X, reduced_Y) dimension
    aggregated = _aggregate(precipitation, n=2, type=aggtype)
    df = aggregated.to_dataframe('RR')
    df['Binary'] = np.where(df['RR'].to_numpy() != 0, 1, 0)
    # NOTE: use of aggregation means this saves coordinates in distances from 0 (where 0 is minimum of each coordinate axis)
    df.to_csv("C:/Users/bened/Documents/University/Cambridge/Thesis/Data/NordicDailyPrecip/Agg({1})_Reduced_{0}.csv".format(str(_to_int(day)), aggtype))

100%|██████████| 365/365 [01:57<00:00,  3.12it/s]
